# HW6: Model-Based RL

In this homework assignment, you will be training a simplified version of the PETS algorithm.
In particular, you will be implementing a version that does not perform ensembling (as all associated operations are much more computationally demanding with ensembling, requiring GPU usage).
In the end, you will be answering a few short questions to help you develop your intuition for model-based methods in general.

**Instructions**: We are no longer requiring you to copy code blocks into a LaTeX template. Instead, simply submit a PDF rendering of this .ipynb file to Gradescope. As usual, we recommend running everything on Google Colab so you will have access to a standardized environment within which this notebook has been completely tested.

If you run into any issues with submission/rendering, please post on Ed or email your preceptor. Furthermore, when submitting, **please double check that all lines are fully visible in the rendered PDF for the code blocks you have written.**

In [ ]:
from typing import Callable, Dict

import gym
import numpy as np
import torch
from torch import nn

## Problem 1: Probabilistic ~~Ensembles~~ with Trajectory Sampling

In this problem, we will be implementing the necessary tools for performing model-based RL, including model definition, model training, and trajectory prediction.
Then, we will put them together to implement a simplified implementation of PETS without ensembling.

### 1(a): Model Definition: Implementing a Neural Network Dynamics Model with Heteroskedastic Noise

For this problem, we will implement a neural network dynamics model with an additive heteroskedastic noise model. Remember that "heteroskedastic" here means that the variance of the additive noise is dependent on the input.

Formally, we want you do define a model that takes in a batch of state-action pairs $(s, a)$, and outputs a batch of Gaussians $\mathcal{N}(s + \mu_\theta(s, a), \Sigma_\theta(s, a))$ representing the distribution of the next state $s'$. **Note that the model predicts the difference between the current and next state!**

* The functions $\mu_\theta$ and $\Sigma_\theta$ should be implemented by a single neural network acting on the input $(s, a)$, whose final output is then partitioned into the mean and covariance components.
* $\Sigma_\theta$ should be a diagonal covariance matrix, and so your architecture should output a vector that will then be used to fill the diagonal. `torch.diag_embed` performs the conversion from a batch of vectors to a batch of diagonal matrices.

Architecture hints:  
* We recommend having two hidden layers of 64 neurons each, with Swish/SiLU activations.
* Use the `scale_tril` constructor argument to parameterize the covariance of `torch.distributions.MultivariateNormal` so that the architecture computes the standard deviation, see the documentation for details.
* The part of the output that will represent the standard deviations should be passed through a softplus (with an added small constant like `1e-9` afterwards) to ensure non-negativity.

You can experiment with the architecture (we will not penalize for using something else), but this should be sufficient for training the agents in this assignment.

In [ ]:
from torch.distributions import Distribution

class NNDynModel(nn.Module):
  def __init__(self, observation_dim, action_dim):
    """Initializes a neural network-based dynamics model with the provided
    observation and action dimensions.

    Args:
      observation_dim: Dimension of the input state and the
        state to be predicted.
      action_dim: Dimension of the input action
    """
    super().__init__()
    self.obs_dim, self.action_dim = observation_dim, action_dim
    #################################
    # Construct model
    self.model = nn.Sequential(
        nn.Linear(observation_dim + action_dim, 64),
        nn.SiLU(),
        nn.Linear(64, 64),
        nn.SiLU(),
        nn.Linear(64, 2 * observation_dim)
    )
    #################################

  def forward(self, observations, actions) -> Distribution:
    """Returns a batch of predicted distribution over the next state, given
    a batch of the current state and a batch of applied actions.

    Args:
      observations: Batch of current states of shape [N, observation_dim].
      actions: Batch of applied actions of shape [N, action_dim].

    Returns:
      A Distribution object representing the predicted distributions over the next
      state for each pair in the batch.
    """
    arch_inputs = torch.concat([observations, actions], dim=-1)
    #################################
    # Compute model output distributions

    arch_outputs = self.model(arch_inputs)


    means = arch_outputs[:, 0:self.obs_dim] + observations
    stddevs = nn.functional.softplus(arch_outputs[:, self.obs_dim:]) + 1e-9
    return torch.distributions.MultivariateNormal(
        loc=means,
        scale_tril=torch.diag_embed(stddevs)
    )
    #################################


### 1(b) Completing a Model Training Loop

Having defined a model, we will now write a function that performs dynamics model training.
We have written most of the boilerplate code for you, including batch sampling; we simply need you to **fill in the indicated code block for performing a single optimization step**.
Assume that we are using maximum likelihood for the loss.

Hint: You should not have to implement the log-likelihood function yourself; check the documentation of `torch.distributions` for a simpler alternative.

In [ ]:
def train_model(
    model: NNDynModel,
    dataset: Dict,
    batch_size: int,
    n_steps: int,
    optimizer: torch.optim.Optimizer
  ) -> NNDynModel:
  """Trains a model by taking the specified number of optimizer steps on the given dataset.

  Args:
    model: Model to train.
    dataset: Dataset to train model on.
    n_steps: Number of optimization steps to take.
    optimizer: An instance of torch.optim.Optimizer for optimizing the given
      model.

  Returns:
    Trained model.
  """
  dataset_size = len(dataset["obs"])

  for _ in range(n_steps):
    batch_idxs = np.random.randint(dataset_size, size=(batch_size))
    batch_ob = torch.Tensor(dataset["obs"][batch_idxs])
    batch_ac = torch.Tensor(dataset["actions"][batch_idxs])
    batch_next_ob = torch.Tensor(dataset["next_obs"][batch_idxs])

    #################################
    # Take an optimization step
    optimizer.zero_grad()
    mean_loss = -torch.mean(model(batch_ob, batch_ac).log_prob(batch_next_ob))
    mean_loss.backward()
    optimizer.step()
    #################################

  return model

### 1(c) Trajectory Prediction

For this problem, you will be implementing a trajectory prediction procedure.
The function below, given a batch of action sequences, returns the model-predicted discounted return for each sequence, where each sequence's prediction is averaged over `preds_per_seq` particles. See the docstring for more details on the input arguments.

To implement multiple particle sampling for each sequence, a trick that we can use is to tile each input action sequence to have `preds_per_seq` copies.
This means that with an input batch of `N` action sequences, we will be performing trajectory prediction for `N * preds_per_seq` sequences simultaneously.
We then use a reshaping trick to take the mean over the particles corresponding to each action sequence.
**Note that we have implemented all this for you**.

**All you need to implement are the updates to the loop variables** (i.e. `cur_states`, `reward_sums`) during this trajectory prediction procedure.
Be aware that `cur_states` is of shape `(N * preds_per_seq, state_dim)`, `reward_sums` is of shape `(N * preds_per_seq)`, and `action_seqs` is of shape `(N * preds_per_seq, sequence_length, 1)`, due to the trick described in the previous paragraph.

In [ ]:
def predict_trajectories(
    model: NNDynModel,
    initial_state: torch.Tensor,
    action_seqs: torch.Tensor,
    preds_per_seq: int,
    reward_fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    discount: float
  ):
  """Returns predicted sum of rewards and final state after applying action_seqs at initial state.
  Averages over preds_per_seq predictions per action sequence, and returns preds_per_seq predictions
  of the final state (not averaged).

  Args:
    model: The model to use for trajectory prediction.
    initial_state: The initial state from which the input action sequences will be applied.
    action_seqs: The action sequences to be evaluated.
    preds_per_seq: The number of particles propagated per action sequence (alternatively,
      the number of samples used to compute an MC estimate of the trajectory return per sequence).
    reward_fn: Reward function used for evaluation. Must take tensors of [batch_size, state_dim]
      and [batch_size, action_dim] (representing the batch states and actions respectively) and
      return a tensor of shape [batch_size] (representing the computed batch rewards).
    discount: Discount to use.

  Returns:
    Estimated discounted returns for each action sequence under the provided model.
  """
  n_seqs = action_seqs.shape[0]
  seq_len = action_seqs.shape[1]

  action_seqs = action_seqs[:, None].tile((1, preds_per_seq, 1, 1)).reshape(n_seqs * preds_per_seq, seq_len, -1)   # Tiling magic
  cur_states = initial_state[None].tile((n_seqs * preds_per_seq, 1))
  reward_sums = torch.zeros(n_seqs * preds_per_seq)

  # At this point, we have that
  # cur_states.shape = [n_seqs * preds_per_seq, state_dim]
  # action_seqs.shape = [n_seqs * preds_per_seq, seq_len, action_dim]
  # reward_sums.shape = [n_seqs * preds_per_seq]
  #
  # Note that the batch dimension is multiplied by preds_per_seq; the trick
  #   we are using here is duplicating everything preds_per_seq times (to create
  #   the particles for each action sequence).
  # At this point, just pretend that you are sampling a single trajectory prediction
  #   for each of the n_seqs * preds_per_seq sequences when filling in the
  #   code block below.

  with torch.no_grad():    # We are not backpropagating through time, speed up by not maintaining gradients
    for t in range(action_seqs.shape[1]):
      #################################
      # Update relevant loop variables
      reward_sums += (discount ** t) * reward_fn(cur_states, action_seqs[:, t])
      cur_states = model(cur_states, action_seqs[:, t]).sample()
      #################################

  reward_sums = reward_sums.reshape((n_seqs, preds_per_seq)).mean(dim=-1)

  return reward_sums

### 1(d) Model Predictive Control

For this problem, **fill in the code block to implement the MPC action selection rule described in lecture**.
Recall that MPC samples a set of action sequences, evaluates each one, then returns only the first action of the best action sequence.
For the action sequence proposal distribution, use a uniform distribution over the box of valid actions for the environment for every action in the sequence. We have defined the variables `center` and `half_width` for your convenience when performing sampling.

Note: **Do not hard code environment-related constants**; for example, if you need the dimension of the action space, you can use relevant properties of `env.action_space` (see the Gym documentation for more information about state/action spaces).

In [ ]:
def create_mpc_policy(env, model, reward_fn, discount, plan_length, n_samples, n_particles):
  """Creates a policy implementing MPC.

  Args:
    env: The environment the created policy will be used for.
    reward_fn: The reward function used by MPC for evaluating action sequences.
    discount: Discount used by MPC for evaluating action sequences.
    plan_length: The length of action sequences considered by MPC.
    n_samples: The number of candidate action sequences sampled by MPC
    n_particles: The number of particles used by MPC to evaluate every action sequence.

  Returns:
    A policy implementing MPC, represented as a function taking in a state and
    returning an action.
  """
  # The action space is a box; center gives the center of the box, while
  #   half_width gives the distance from the center to either end for each dimension.
  # Ex. for an action space [-n, n]^d, center = zero vector in d dimensions,
  #   half_width = vector of all n's in d dimensions.
  center = torch.Tensor((env.action_space.high + env.action_space.low) / 2)
  half_width = torch.Tensor((env.action_space.high - env.action_space.low) / 2)

  def mpc_policy(state: np.ndarray) -> np.ndarray:
    """Returns the action taken by MPC on the given state.

    Args:
      state: Input state to policy of shape [state_dim].

    Returns:
      Action computed by MPC.
    """
    state = torch.Tensor(state)
    #################################
    # Implement MPC action selection rule
    # Hint: Remember to sample actions within each sequence uniformly within the action space
    #   (use the center and half_width variables we define above!)
    prescaled_seqs = 2 * (torch.rand((n_samples, plan_length, env.action_space.shape[0])) - 0.5)  # Uniform on [-1, 1]^action_dim
    action_seqs = half_width * prescaled_seqs + center
    reward_sums = predict_trajectories(model, state, action_seqs, n_particles, reward_fn, discount)
    return action_seqs[torch.argmax(reward_sums)][0].numpy()
    #################################

  return mpc_policy

### 1(e) PETS Training Loop

Finally, we will put all of the components we have implemented so far together.
**Fill in the indicated code blocks in `train` to complete the PETS training loop for the `Pendulum-v1` environment in Gym**. Here is a brief description of each code block:

Code Block 1: Initialize the necessary objects for training (model, policy, model training buffer, and optimizer). We recommend AdamW, with weight decay set to 0.1.  
Code Block 2: Implement a uniform random policy for selecting actions during exploration episodes (Hint: see `env.action_space` for potentially useful methods).  
Code Block 3: Obtain an action from an MPC-based policy.  
Code Block 4: Add transition data to the model training buffer.  
Code Block 5: Continue training the model.

We have provided a `MBRLBuffer` class for you; the `get_training_dataset()` method returns an object that is compatible with the `train_model()` method you had implemented earlier.
Furthermore, `pendulum_reward()` computes the same reward function as the `Pendulum-v1` environment itself, and can be used during MPC evaluation.

**Note on expected returns**: Due to the randomness of the initial state, the return for this environment is highly variable.
For debugging reasons, we have included a print statement which prints the average absolute deviation of the pendulum angle from zero for the last 100 timesteps of each rollout, in addition to the return.
Since the objective is to balance the pendulum rod ($\theta = 0$), your implementation can be said to have learned `Pendulum-v1` if the printed average $\theta$ deviations are very close to zero.

**Note on runtime**: Our implementation on Google Colab takes around five minutes to run five episodes of MPC, which is sufficient for learning the task (if not less).

**Note on constants**: All necessary constants (e.g. number of MPC particles, number of candidate action sequences, etc.) are provided for you already as optional arguments with default values.


In [ ]:
class MBRLBuffer:
  def __init__(self, observation_dim, action_dim, buffer_size):
    self.buffer_size = buffer_size

    self.ptr = 0
    self.length = 0
    self.ob_buffer = np.zeros((buffer_size, observation_dim))
    self.action_buffer = np.zeros((buffer_size, action_dim))
    self.next_ob_buffer = np.zeros((buffer_size, observation_dim))

  def add_datapoint(self, observation, action, next_observation):
    self.ob_buffer[self.ptr] = observation
    self.action_buffer[self.ptr] = action
    self.next_ob_buffer[self.ptr] = next_observation
    self.ptr = (self.ptr + 1) % self.buffer_size
    self.length = min(self.length + 1, self.buffer_size)

  def get_training_dataset(self):
    return {
        "obs": self.ob_buffer[:self.length],
        "actions": self.action_buffer[:self.length],
        "next_obs": self.next_ob_buffer[:self.length]
    }

In [ ]:
def pendulum_reward(states: torch.Tensor, actions: torch.Tensor) -> torch.Tensor:
  return -(torch.square(torch.atan2(states[:, 1], states[:, 0])) + 0.1 * torch.square(states[:, 2]) + 0.001 * torch.square(actions[:, 0]))

def train_pendulum(
    n_episodes=10,
    n_exploration_episodes=1,
    discount=0.99,
    mpc_plan_length=15,
    n_mpc_candidate_sequences=1000,
    n_particles=10,
    buffer_length=10000,
    model_batch_size=32,
    n_model_steps=512
  ) -> None:
  """Trains PETS on the Pendulum-v1 environment.

  Args:
    n_episodes: Number of episodes collected using MPC
    n_exploration_episodes: Number of initial episodes collected using
      uniform random actions.
    discount: Discount used by MPC and for evaluation.
    mpc_plan_length: Length of action sequences considered by MPC.
    n_mpc_candidate_sequences: Number of candidate sequences considered by MPC
      during the selection process.
    n_particles: Number of particles used by MPC per action sequence during evaluation.
    buffer_length: Length of model training buffer.
    model_batch_size: Batch size during model training.
    n_model_steps: Number of optimization steps taken every time the model is
      trained after an episode.

  Returns:
    None
  """
  env = gym.make("Pendulum-v1")
  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.shape[0]

  #################################
  # Create model, policy, training buffer, and optimizer
  model = NNDynModel(state_dim, action_dim)
  policy = create_mpc_policy(
    env, model, pendulum_reward, discount, mpc_plan_length, n_mpc_candidate_sequences, n_particles
  )
  buffer = MBRLBuffer(state_dim, action_dim, buffer_length)
  optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.1)
  #################################


  for episode in range(n_exploration_episodes + n_episodes):
    reward_sum = 0
    deviations = []
    state = env.reset()
    done = False
    timestep = 0

    while not done:
      if episode < n_exploration_episodes:
        #################################
        # Select random action for initial exploration episodes
        action = env.action_space.sample()
        #################################
      else:
        #################################
        # Use MPC to get an action.
        action = policy(state)
        #################################

      next_state, reward, done, _ = env.step(action)
      reward_sum += (discount ** timestep) * reward
      deviations.append(np.arctan2(next_state[1], next_state[0]))

      #################################
      # Add transition data to replay buffer
      buffer.add_datapoint(state, action, next_state)
      #################################

      timestep += 1
      state = next_state

    if episode >= n_exploration_episodes:
      print("Episode {} return: {}; average deviation over final 100 steps: {}".format(
          episode - n_exploration_episodes, reward_sum, np.mean(deviations[-100:])
      ))

    #################################
    # Continue training the model
    train_model(model, buffer.get_training_dataset(), model_batch_size, n_model_steps, optimizer)
    #################################

In [ ]:
train_pendulum()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0 return: -302.5487001707376; average deviation over final 100 steps: 0.09798536449670792
Episode 1 return: -2.001051440750063; average deviation over final 100 steps: 0.010249728336930275
Episode 2 return: -195.77209295631263; average deviation over final 100 steps: -0.0002570024225860834
Episode 3 return: -199.207247786024; average deviation over final 100 steps: 0.003300018608570099
Episode 4 return: -103.58109111925118; average deviation over final 100 steps: 0.001337804482318461
Episode 5 return: -281.3060626422867; average deviation over final 100 steps: 0.00436615664511919
Episode 6 return: -110.0614368586988; average deviation over final 100 steps: -0.007343442644923925
Episode 7 return: -106.28333708455487; average deviation over final 100 steps: 0.002810687990859151
Episode 8 return: -208.71882428577678; average deviation over final 100 steps: 0.009274815209209919
Episode 9 return: -201.82953923556136; average deviation over final 100 steps: 0.01171807385981083


## Problem 2: Short Answers

Please keep your responses brief; **most of the justifications for these questions can be answered in one or two sentences**.

### 2(a) MPC Horizon

One of the hyperparameters that needs to be chosen for MPC is the length of the action sequences that are evaluated at every step (i.e. the planning horizon). We have listed four factors that should be considered when choosing this hyperparameter. Explain **how each consideration affects the choice of MPC horizon and why**.

#### (i) Action sequence optimization difficulty

> Sampling a good action sequence gets harder with longer sequences due to the curse of dimensionality, and so this would encourage us to choose shorter sequences.



#### (ii) Achieved returns

> Since we are ultimately trying to solve the RL problem, we want the evaluation metric to reflect the full return as well as possible, hence we would want the horizon to be longer.


#### (iii) Model accuracy

> The worse the model accuracy, the harder it is to get a reliable trajectory prediction for long action sequences.

#### (iv) Wall-clock-time, running on a non-simulated system

> Trajectory prediction is computationally expensive with respect to the horizon because the runtime is linear in the horizon, hence real systems may require a shorter horizon for responsive control.

### 2(b) Exploration Bonuses with Ensembles, Part I

Consider an agent who is running MPC, but is evaluating each action sequence differently than the procedure discussed in lecture. More concretely, for a particular action sequence, let us say that the sampled returns computed from particle-based sampling is given by $(R_{i, j})$ for $i = 1, \dots, M$ and $j = 1, \dots, N$, such that $R_{i, 1}, R_{i, 2}, \dots, R_{i, N}$ are the predictions from the $i^{\text{th}}$ model of the ensemble.
For convenience, let us define the within-model mean $\mu_i$ and within-model standard deviation $\sigma_i$ for $i = 1, \dots, M$ as

$$\mu_i := \frac{1}{N}\sum_{j = 1}^{N}R_{i, j} \quad \text{and} \quad \sigma_i := \sqrt{\frac{1}{N - 1}\sum_{j = 1}^{N}(R_{i, j} - \mu_i)^2}$$

The standard MPC evaluation procedure computes the quantity

$$\hat{R}_{\mathrm{basic}} := \frac{1}{M}\sum_{i = 1}^{M}\mu_i,$$

which is equivalently the average over all particles.
This agent instead computes the evaluation metric

$$\hat{R}_{\mathrm{proposed}} := \frac{1}{M}\sum_{i = 1}^{M}(\mu_i + \sigma_i)$$

to add some "exploration bonus" by encouraging the agent to go where there is more observed noise (analogous to the UCB bonus for bandits).
**Does this bonus make sense as an exploration bonus? Why or why not?**

Hint: Think about the fact that the standard deviations are *within-model* standard deviations. Where does the noise in within-model predictions come from?

> This bonus does not make sense, since the variance within each model's predictions is due to irreducible environment noise, and so this objective would encourage the agent to go to noisy parts of the environment.

### 2(c) Exploration Bonuses with Ensembles, Part II

We work in the same setting as 3(b), but now let us define the across-ensemble standard deviation $\sigma_{\mathrm{Ens}}$ as the standard deviation of the within-model means, or written mathematically,

$$\sigma_{\mathrm{Ens}} := \sqrt{\frac{1}{M - 1}\sum_{i = 1}^{M}(\mu_i - \hat{R}_\mathrm{basic})^2}.$$

The agent now uses the evaluation metric

$$\hat{R}_{\mathrm{proposed}} := \hat{R}_{basic} + \sigma_{\mathrm{Ens}}$$

as another form of an "exploration bonus".
Does this new bonus make sense as an exploration bonus? Why or why not?

> This makes sense as a bonus, since the variance is across the bootstrapped ensemble, which is an estimate of the epistemic uncertainty of the model that can be addressed with more data. Thus, the agent would be encouraged to take actions that will help it improve its return estimates.

### 2(d) What to use?

We will describe several scenarios for which we might want to use RL.
In each case, **state whether a model-based or a model-free algorithm would be preferred and why**.

1. Training an expensive but fragile robotic arm for reaching tasks.

> Model-based. Using a model is key in real environments where available data is limited by the system.

2. Training an agent to play a video game that can be run massively in parallel.

> Model-free. If getting data is cheap and easy, then there is no reason to use a model, as the model can limit performance.

3. Training an RC car to deliver food to college students anywhere on campus.

> Model-based. One can think of each possible student location as a new task represented by a different reward function, but the dynamics remains the same so the same model can be used across all these tasks.

### 2(e) Terminal Value Functions

Observe that the standard MPC evaluation procedure only considers the sum of rewards within the action sequence, and does not consider anything after the sequence.
One can instead add a terminal value function at the end that is learned alongside the model throughout the training procedure.
**Name one pro and one con of learning and using this value function.**

Hint: for the drawback, think about the number of samples needed to learn a value function.

> **Pro:** Selected action sequence will maximize the full return rather than just the sum of returns for the action sequence, removing the mismatch with the true RL objective.

> **Con:** Learning a value function requires more samples, minimizing the benefit of using a model-based approach.